# Import Libs

In [283]:
import re
import os
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
# import community
import nxviz as nv
from networkx.algorithms.community import girvan_newman, louvain_communities, louvain_partitions

# Extraindo Personagens

## Descobrindo nomes arquivos de entrada

In [284]:
def list_path_temporadas() -> list:
    dir_dados = 'data'
    diretorios = os.listdir(dir_dados)
    diretorios_temporadas = [f'{dir_dados}/{f}' for f in diretorios if os.path.isdir(f'{dir_dados}/{f}')]
    return diretorios_temporadas

In [285]:
def list_path_epi(path_temporada) -> list:
    diretorios = os.listdir(path_temporada)
    path_episodios = [f'{path_temporada}/{f}' for f in diretorios if os.path.isfile(f'{path_temporada}/{f}')]
    return path_episodios

# Extrair Personagens

In [286]:
def regex_nome_personagem(string):
    string_upper = string.upper()
    regex_result = re.search('^[^"\[]*?:', string_upper)
    if regex_result:
        string_personagem = regex_result.group(0)
        if re.search('((^EXT)|(^INT))', string_upper):
            return None
        return regex_result
    # return re.search('^[^"\[]*?:', strin_upper)

In [287]:
def tratamento_nome_personagem(personagem_str):
    p = re.compile('((\(.*\))*)(\s?):')
    personagem_str = p.sub('', personagem_str) 
    p = re.compile('\s$')
    personagem_str = p.sub('', personagem_str)
    
    if 'YOUNG ' in personagem_str:
        return personagem_str.replace('YOUNG ', '')
    elif ' VOICE' in personagem_str:
        return personagem_str.replace('\'S VOICE', '')    
    
    return personagem_str

In [288]:
def contem_digito(personagem_str):
    return any(char.isdigit() for char in personagem_str)

In [289]:
def extrair_personagens_episodio(path_arquivo, personagens, personagens_figurantes):
    with open(path_arquivo, 'r') as file:
        for line in file:
            personagem = regex_nome_personagem(line)

            if personagem:
                personagem_str = tratamento_nome_personagem(personagem.group(0))
                
                contains_digit = any(char.isdigit() for char in personagem_str)
                
                if contains_digit:
                    p = re.compile('(\s?).(\d)')
                    personagem_figurante = p.sub('', personagem_str)
                    personagens_figurantes.add(personagem_figurante)
                else:
                    x = re.search('((^ALL\s)|(^ALL$)|(^AN?\s))', personagem_str)
                    if not x:
                        personagens.add(personagem_str)

In [290]:
def personagem_invalido(personagem_str):
    return ',' in personagem_str or '.' in personagem_str or 'CUT TO' in personagem_str or 'VOICE ' in personagem_str

In [291]:
def extrair_personagens():
    personagens = set()
    
    personagens_figurantes = set()
    
    for path_temporada in list_path_temporadas():
        for path_episodio in list_path_epi(path_temporada):
            extrair_personagens_episodio(path_episodio, personagens, personagens_figurantes)
            
    personagens_principais = set()
    for personagem in personagens:
        if personagem_invalido(personagem):
            continue
             
        if personagem not in personagens_figurantes:
            personagens_principais.add(personagem)    
             
    return personagens_principais

In [292]:
personagens = extrair_personagens()

# Divisão Cenas

In [293]:
def linha_atual_dialogo(personagem_regex):
    return False if not personagem_regex else True
    

In [294]:
def personagens_na_mesma_cena(dialogos, personagens):
    personagens_cena = set()
    
    for dialogo in dialogos:
        nome_personagem = regex_nome_personagem(dialogo).group(0)
        nome_personagem = tratamento_nome_personagem(nome_personagem)
        
        if personagem_invalido(nome_personagem):
            continue
        
        if nome_personagem in personagens:    
            personagens_cena.add(nome_personagem)
        
    return personagens_cena

In [295]:
def nova_cena(ultima_linha_dialogo, linha_atual_contem_dialogo, dialogo_iniciado_capitulo, line):
    if 'scene' in line:
        return True
    if '- - -' in line:
        return True
    if re.search('((^EXT)|(^INT))', line):
        return True
    # if re.search('^INT', line):
    #     return True
    # if 'EXT.' in line: 
    #     return True
    # if 'INT.' in line:
    #     return True
    # if 'EXT-' in line:
    #     return True
    # if 'INT' in line:
    #     return True
    if re.search('^CUT TO', line):
        return True
    if not ultima_linha_dialogo and dialogo_iniciado_capitulo:
        if linha_atual_contem_dialogo:
            return True
    return False
            

In [296]:
def identificacao_cenas(personagens):
    personagens_cenas_serie = []
    for path_temporada in list_path_temporadas():
        for path_episodio in list_path_epi(path_temporada):
            
            with open(path_episodio, 'r') as file:
                dialogos = []
                ultima_linha_dialogo = False
                gerar_quebra_cenario = False
                dialogo_iniciado_capitulo = False
                for line in file:
                    
                    if line == '\n':
                        continue
                    
                    personagem_regex = regex_nome_personagem(line)

                    linha_atual_contem_dialogo = linha_atual_dialogo(personagem_regex)                   

                    if personagem_regex:
                        dialogo_iniciado_capitulo = True
                        dialogos.append(line)
                    elif nova_cena(ultima_linha_dialogo, linha_atual_contem_dialogo, dialogo_iniciado_capitulo, line):
                        personagens_cena = personagens_na_mesma_cena(dialogos, personagens)
                        if len(personagens_cena) > 1:
                            personagens_cenas_serie.append(personagens_cena)
                        dialogos = []
                    
                    ultima_linha_dialogo = linha_atual_contem_dialogo         
                        
    return personagens_cenas_serie                    

In [297]:
personagens_cenas = identificacao_cenas(personagens)

# Gerar Conexões dos Personagens

In [298]:
def extrair_personagens_recorrentes(personagens_cenas, frequencia_minima):
    frequencia_personagem = {}
    for cena in personagens_cenas:
        for personagem in cena:
            if personagem not in frequencia_personagem.keys():
                frequencia_personagem[personagem] = 0
            frequencia_personagem[personagem] += 1
            
    for key in frequencia_personagem.keys():
        frequencia_personagem[key] /= len(personagens_cenas)
            
    personagens_recorrentes = set()
    for personagem, frequencia in frequencia_personagem.items():
        if frequencia >= frequencia_minima:
            personagens_recorrentes.add(personagem)
            
    return personagens_recorrentes

In [299]:
def extrair_dados_grafo_personagens(persoangens, personagens_cenas):    
    personagens_recorrentes = extrair_personagens_recorrentes(personagens_cenas, 0.01)
    
    indice_personagem = 0
    map_personagem = {}
    for personagem in personagens:
        if personagem in personagens_recorrentes:
            map_personagem[personagem] = indice_personagem
            indice_personagem += 1
            
    conexoes_personagens = {}
    
    for personagens_cena in personagens_cenas:
        personagens_cena_list = list(personagens_cena)
        for i in range(0, len(personagens_cena)):
            if personagens_cena_list[i] not in personagens_recorrentes:
                continue
            for j in range(i+1, len(personagens_cena)):
                if personagens_cena_list[j] not in personagens_recorrentes:
                    continue

                personagem_1 = min(personagens_cena_list[i], personagens_cena_list[j])
                personagem_2 = max(personagens_cena_list[i], personagens_cena_list[j])
                
                if personagem_1 not in conexoes_personagens.keys():
                    conexoes_personagens[personagem_1] = {}
                    
                if personagem_2 not in conexoes_personagens[personagem_1].keys():
                    conexoes_personagens[personagem_1][personagem_2] = 0
                    
                conexoes_personagens[personagem_1][personagem_2] += 1

    conexoes = []
    
    for p1 in conexoes_personagens.keys():
        if p1 == 'NAN': 
            continue
        for p2 in conexoes_personagens[p1].keys():
            if p2 == 'NAN': 
                continue
            peso = conexoes_personagens[p1][p2]/len(personagens_cenas)
            cenas = conexoes_personagens[p1][p2]
            
            conexoes.append({'p1': p1, 'p2': p2, 'peso': peso, 'cenas': cenas})

    return pd.DataFrame.from_dict(conexoes)

    # dados_grafo = {'conexoes_personagens': conexoes_personagens, 'personagens': map_personagem, 'num_cenas': len(personagens_cenas)}
    # return dados_grafo

## Criação do grafo

In [300]:
df = extrair_dados_grafo_personagens(personagens, personagens_cenas)

In [301]:
G = nx.from_pandas_edgelist(
    df, source='p1', target='p2',
    edge_attr=['peso', 'cenas']
)

## Medidas de Centralidade

### Centralidade

In [302]:
centralidade = nx.degree_centrality(G)

In [303]:
centralidade = sorted(centralidade.items(), key=lambda x:x[1], reverse=True)

In [304]:
centralidade[0:10]

[('JON', 0.6707317073170732),
 ('ARYA', 0.6707317073170732),
 ('SANSA', 0.6585365853658537),
 ('TYRION', 0.5975609756097561),
 ('JAIME', 0.5853658536585367),
 ('DAENERYS', 0.5487804878048781),
 ('BRAN', 0.5365853658536586),
 ('CERSEI', 0.5365853658536586),
 ('JORAH', 0.524390243902439),
 ('BRIENNE', 0.5121951219512195)]

### Centralidade Autovetor

In [305]:
centralidade_autovetor = nx.eigenvector_centrality(G)


In [306]:
centralidade_autovetor = sorted(centralidade_autovetor.items(), key=lambda x:x[1], reverse=True)
centralidade_autovetor[0:10]

[('SANSA', 0.20584913079209485),
 ('ARYA', 0.2026195002009149),
 ('JON', 0.2026049164233179),
 ('TYRION', 0.19422822983564716),
 ('JAIME', 0.1869163535011763),
 ('DAENERYS', 0.18266995963261803),
 ('JORAH', 0.1789197702585332),
 ('BRAN', 0.17507487845463043),
 ('DAVOS', 0.17295956847969104),
 ('THEON', 0.1696598107877602)]

### Centralidade Pagerank

In [307]:
pagerank = nx.pagerank(G)


In [308]:
pagerank = sorted(pagerank.items(), key=lambda x:x[1], reverse=True)
pagerank[0:10]

[('ARYA', 0.02583193813454142),
 ('JON', 0.025353922326544143),
 ('SANSA', 0.024288193800858063),
 ('TYRION', 0.021940619220889288),
 ('JAIME', 0.021936591409387973),
 ('DAENERYS', 0.020657753074910915),
 ('BRAN', 0.020148255258087642),
 ('VARYS', 0.020081277025064886),
 ('CERSEI', 0.020039980592030333),
 ('BRIENNE', 0.019798755861106427)]

### Centralidade Proximidade

In [309]:
centralidade_proximidade = nx.closeness_centrality(G)


In [310]:
centralidade_proximidade = sorted(centralidade_proximidade.items(), key=lambda x:x[1], reverse=True)
centralidade_proximidade[0:10]

[('JON', 0.7522935779816514),
 ('ARYA', 0.7522935779816514),
 ('SANSA', 0.7454545454545455),
 ('TYRION', 0.7130434782608696),
 ('JAIME', 0.7068965517241379),
 ('BRAN', 0.6833333333333333),
 ('CERSEI', 0.6833333333333333),
 ('DAENERYS', 0.6833333333333333),
 ('JORAH', 0.6721311475409836),
 ('DAVOS', 0.6721311475409836)]

## Detecção de Comunidades

In [311]:
# partition = community.best_partition(G, randomize=False)

comunidades = louvain_communities(G, weight='peso')


In [312]:
len(comunidades)

7

In [313]:
print(type(comunidades))

<class 'list'>


In [323]:
pos = nx.spring_layout(G)

# nx.draw(G, pos, edge_color='blue', node_color='red', with_labels=False,
#          font_weight='light', node_size= 10, width= 0.9)

# centralidade_autovetor

d = nx.degree(G)

node_key = [item[0] for item in centralidade_autovetor]

node_values = [item[1] * 500 for item in centralidade_autovetor]

peso_personagens = {}
for item in centralidade_autovetor:
    peso_personagens[item[0]] = item[1]

colors_list = ['pink', 'green', 'blue', 'red', 'orange', 'yellow', 'brown']

color_edges = []

for personagem in node_key:
    for i in range(0, len(comunidades)):
        if personagem in comunidades[i]:
            color_edges.append(colors_list[i])
            break

nx.draw(G, pos=pos, nodelist=node_key, node_size=node_values, edge_color='k', font_weight='light')

for i in range(0, len(comunidades)):
    node_values = [peso_personagens[personagem] * 500 for personagem in comunidades[i]]
    nx.draw_networkx_nodes(G, pos, nodelist=comunidades[i], node_color=colors_list[i], edgecolors=colors_list[i], node_size=node_values)


In [315]:
# for n in G.nodes():
#     G.nodes[n]["partition"] = partition[n]
#     G.nodes[n]["degree"] = G.degree(n)

In [316]:
# partition_dict = {}
# for character, par in partition.items():
#     if par in partition_dict:
#         partition_dict[par].append(character)
#     else:
#         partition_dict[par] = [character]

In [317]:
def most_important_node_in_partition(graph, partition_dict):
    max_d = {}
    deg = nx.degree_centrality(graph)
    for group in partition_dict:
        temp = 0
        for character in partition_dict[group]:
            if deg[character] > temp:
                max_d[group] = character
                temp = deg[character]
    return max_d

In [318]:
most_important_node_in_partition(G, partition)

TypeError: list indices must be integers or slices, not set

In [ ]:
# nv.matrix(G, group_by="partition", sort_by="degree", node_color_by="partition")
